In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [48]:
df = pd.read_csv("data/medical_insurance_scaled.csv", sep=",")
df.head()
df.isna().sum()

age          0
sex          0
bmi          0
children     0
smoker       0
charges_x    0
northeast    0
northwest    0
southeast    0
charges_y    0
dtype: int64

In [45]:
df.charges.describe()

count     1335.000000
mean     13275.682489
std      12107.717986
min       1121.873900
25%       4746.698450
50%       9386.161300
75%      16622.107580
max      63770.428010
Name: charges, dtype: float64

In [46]:
# On tente de target (charges) par tranche de 5000 jusqu'à 100000 en les label encodant 
df['charges'] = pd.cut(df['charges'], bins=range(0, 100000, 5000), right=False)
le = LabelEncoder()
df['charges'] = le.fit_transform(df['charges'])

In [47]:
from sklearn.cluster import KMeans


X = df.drop('charges', axis=1)
y = df['charges']

charges
0     357
1     353
2     268
3      85
4      72
7      54
8      41
5      38
9      31
6      29
12      3
10      2
11      2
13      1
Name: count, dtype: int64